In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

In [ ]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

In [45]:
# Load the data
interpolated_dir = os.path.join(parent_dir, "dataset_interpolated")
def load_data(interpolated_dir):
    data = {}
    for folder in os.listdir(interpolated_dir):
        if folder != "2019":
            continue
        folder_path = os.path.join(interpolated_dir, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith(".csv"):
                    file_path = os.path.join(folder_path, file)
                    df = pd.read_csv(file_path)
                    for _, row in df.iloc[1:].iterrows():
                        row["relative_strength"] = df.iloc[0]["homeWinProbability"]
                        row["away_team_id"] = df.iloc[0]["away_team_id"]
                        row["home_team_id"] = df.iloc[0]["home_team_id"]
                        row["home_win"] = df.iloc[0]["home_win"]
                        if row["timestep"] not in data:
                            data[row["timestep"]] = [row] 
                        else:
                            data[row["timestep"]] += [row]
    return data

data = load_data(interpolated_dir)

In [52]:
# Feature Selection
def feature_selection(data, features):
    # Given the features of the data, return data such that each row is an array of the values of the features
    # The data is a dictionary where the key is the timestep and the value is a list of rows
    feature_data = {}
    for timestep in data:
        feature_data[timestep] = []
        for row in data[timestep]:
            feature_data[timestep] += [[float(row[feature]) for feature in features]]
    return feature_data

features = ["home_win", "relative_strength", "homeScore", "awayScore", "scoringPlay", "start.down", "start.distance", "start.yardLine", "end.down", "end.distance", "end.yardLine"]
features_data = feature_selection(data, features)

In [ ]:
# Setup model for each timestep
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score